In [1]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = "1xnMYOD5FkrwVKakAAaAaqOyD"
        consumer_secret = "hf1ADLyEPWll1rl43sGHdwl7eMWmK1LbzU6ajSxQKbwB86I15D"
        access_token = "732181352326434816-XpBDVRPACxdBtChDFqxVhH2bz5nRGm0"
        access_token_secret = "2tu6wcnVBTy6aZNtNjYrSmzzJv1pAvGI2YbTNgjXPc8rv"
        

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return (' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()))

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))




In [8]:
import pandas as pd

def tweetListMaker(tweetsList,sentimentNumber):
    processedTweetsList = []
    for i in tweetsList:
        #print("Positive")

        #print(type(i),i)
        if isinstance(i,dict):
            processedTweetsList.append([i['text'],sentimentNumber])
        elif isinstance(i,list):
            for j in i:
                if isinstance(i,dict):
                    processedTweetsList.append([j['text'],sentimentNumber])
    
    return (processedTweetsList)
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    
    myFilters = ["cash flow", "balance sheet", "stock", "prices", "deprecation","rich"]
    ptweets = []
    ntweets = []
    neuTweets = []
    
    myDf = pd.DataFrame()
    for currentFilter in myFilters:
        # calling function to get tweets
        tweets = api.get_tweets(query = currentFilter, count = 40000)

        print(tweets)
        # picking positive tweets from tweets
        print(len(ptweets),len(ntweets),len(neuTweets))
        if len(ptweets) == 0:
            ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
        else:
            ptweets.append([tweet for tweet in tweets if tweet['sentiment'] == 'positive'])
        # percentage of positive tweets
        print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
        # picking negative tweets from tweets
        
        if len(ntweets) == 0:
            ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
        else:
            ntweets.append([tweet for tweet in tweets if tweet['sentiment'] == 'negative'])
        # percentage of negative tweets
        print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))

        neut = [tweets not in ptweets]
        
        if len(neuTweets) == 0:
            neuTweets = [neut not in ntweets]
        else:
            neuTweets.append([neut not in ntweets])
        
        print(len(neuTweets))
        # percentage of neutral tweets
        #print("Neutral tweets percentage: {} %".format(100*len(tweets - ntweets - ptweets)/len(tweets)))

        # printing first 5 positive tweets
        #print("\n\nPositive tweets:")
        #for tweet in ptweets[:10]:
        #    print(tweet['text'])

        # printing first 5 negative tweets
        #print("\n\nNegative tweets:")
        #for tweet in ntweets[:10]:
        #    print(tweet['text'])
    
    a = [t for t in ptweets]
    #print(a)
    #myDf = pd.DataFrame([[t['text'],3] for t in ptweets],columns=['tweet','label'])
    #myDf = myDf.append(pd.DataFrame([[t['text'],3] for t in ntweets],columns=['tweet','label']))
    #myDf = myDf.append(pd.DataFrame([[t['text'],3] for t in neuTweets],columns=['tweet','label']))
    pt = tweetListMaker(ptweets,3)
    nt = tweetListMaker(ptweets,1)
    neut = tweetListMaker(neuTweets,2)
    #myDf['label'].iloc[currentLen:]  = 2
    
    myDf = pd.DataFrame(pt,columns=['tweet','label'])
    myDf = myDf.append(pd.DataFrame(nt,columns=['tweet','label']))
    myDf = myDf.append(pd.DataFrame(neut,columns=['tweet','label']))
    currentLen = len(myDf)
    
    
    return (myDf)


#print(ptweets)
#myDf['tweets'] = ptweets
#myDf['label']  = 3


myDf = main()


[{'sentiment': 'neutral', 'text': '#thursdaythoughts https://t.co/ugGzTw3lgT Time to get your business space organised for maximum cash flow https://t.co/nIpF0UPlUK'}, {'sentiment': 'positive', 'text': '#AstonMartin unveils new #Vantage in plan to set up possible #StockMarket flotation. "It\'s important fiscally becau… https://t.co/IKcH18UOJl'}, {'sentiment': 'neutral', 'text': "RT @JohnZettler: Why does no one talk about how high-beta @WeWork's biz model is? End customers are freelancers &amp; early-stage startups (bac…"}, {'sentiment': 'positive', 'text': "Register for today's free webinar! If you are a pro #photographer looking to offer a portrait experience that will… https://t.co/v0CQgOPZU5"}, {'sentiment': 'negative', 'text': '4 smart cash flow tips for owners of small rental properties—and why cash flow matters https://t.co/eNomDsOFNO via @Buildium'}, {'sentiment': 'positive', 'text': 'I wish I had the energy to do things, im so worn out! Gotta find a new job, write a book, lear

In [7]:
myDf[myDf.label == 3.0]

,tweet,label
0,#AstonMartin unveils new #Vantage in plan to s...,3.0
1,Register for today's free webinar! If you are ...,3.0
2,"I wish I had the energy to do things, im so wo...",3.0
3,.@MastercardNews finds SMBs are using cards mo...,3.0
4,"RT @ForgeRiver: At Current Prices, Whiting Pet...",3.0
5,RT @theRealKiyosaki: Find investments that cas...,3.0
6,"MindStar Aviation achieved ""100% consistency i...",3.0
7,Duplex in Glendale with great cash flow\n\n🏡68...,3.0
8,RT @AiMMoves: #XTR \n\n🌟$300k (net) per month ...,3.0
9,RT @TrinityAssetMan: DeutscheBank was first se...,3.0


In [2]:
!pip install tweepy
!pip install textblob

  Using cached textblob-0.13.1-py2.py3-none-any.whl


In [3]:
myDf['tweets'].iloc[0]

NameError: name 'myDf' is not defined

In [56]:
len(myDf)

32

In [61]:
pd.DataFrame({"sahil",2},columns=['tweets','label'])

,tweets,label
0,sahil,2
